# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

2022-03-25 10:04:47.208595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-25 10:04:47.242603: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-25 10:04:47.242620: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-25 10:04:47.243206: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

"labels = ['label:LYING_DOWN',\n 'label:SITTING',\n 'label:FIX_walking',\n 'label:FIX_running',\n 'label:BICYCLING',\n 'label:SLEEPING',\n 'label:LAB_WORK',\n 'label:IN_CLASS',\n 'label:IN_A_MEETING',\n 'label:LOC_main_workplace',\n 'label:OR_indoors',\n 'label:OR_outside',\n 'label:IN_A_CAR',\n 'label:ON_A_BUS',\n 'label:DRIVE_-_I_M_THE_DRIVER',\n 'label:DRIVE_-_I_M_A_PASSENGER',\n 'label:LOC_home',\n 'label:FIX_restaurant',\n 'label:PHONE_IN_POCKET',\n 'label:OR_exercise',\n 'label:COOKING',\n 'label:SHOPPING',\n 'label:STROLLING',\n 'label:DRINKING__ALCOHOL_',\n 'label:BATHING_-_SHOWER',\n 'label:CLEANING',\n 'label:DOING_LAUNDRY',\n 'label:WASHING_DISHES',\n 'label:WATCHING_TV',\n 'label:SURFING_THE_INTERNET',\n 'label:AT_A_PARTY',\n 'label:AT_A_BAR',\n 'label:LOC_beach',\n 'label:SINGING',\n 'label:TALKING',\n 'label:COMPUTER_WORK',\n 'label:EATING',\n 'label:TOILET',\n 'label:GROOMING',\n 'label:DRESSING',\n 'label:AT_THE_GYM',\n 'label:STAIRS_-_GOING_UP',\n 'label:STAIRS_-_GOING

In [3]:
labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]

len(labels)

6

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [4]:
#k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')
k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- taxa de aprendizado (*learning rate*) do otimizador (*Adaptive Moment Estimation*, **Adam**): 1e-1, 1e-2, 1e-3 ou 1e-4

In [5]:

for fold in k_folds:
    config = {
        'df_path': k_folds[fold]['40'],
        'labels': labels
    }
    
    har = utils.HAR(config)
    #har.run()
    har.hypertunning()
    
    har.mlp.model.save(f'model/saved_model_{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'model/saved_model_{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
3922/3922 [==============================] - 5s 976us/step - loss: 0.1975 - avg_multilabel_BA_2: 0.7329 - val_loss: 0.1723 - val_avg_multilabel_BA_2: 0.7873
Epoch 2/100
3922/3922 [==============================] - 3s 886us/step - loss: 0.1629 - avg_multilabel_BA_2: 0.7998 - val_loss: 0.1581 - val_avg_multilabel_BA_2: 0.8080
Epoch 3/100
3922/3922 [==============================] - 4s 943us/step - loss: 0.1538 - avg_multilabel_BA_2: 0.8133 - val_loss: 0.1523 - val_avg_multilabel_BA_2: 0.8178
Epoch 4/100
3922/3922 [==============================] - 4s 902us/step - loss: 0.1485 - avg_multilabel_BA_2: 0.8211 - val_loss: 0.1473 - val_avg_multilabel_BA_2: 0.8242
Epoch 5/100
3922/3922 [==============================] - 4s 898us/step - loss: 0.1455 - av

Epoch 48/100
3922/3922 [==============================] - 3s 883us/step - loss: 0.1242 - avg_multilabel_BA_2: 0.8596 - val_loss: 0.1293 - val_avg_multilabel_BA_2: 0.8597
Epoch 49/100
3922/3922 [==============================] - 4s 946us/step - loss: 0.1239 - avg_multilabel_BA_2: 0.8598 - val_loss: 0.1305 - val_avg_multilabel_BA_2: 0.8599
Epoch 50/100
3922/3922 [==============================] - 4s 951us/step - loss: 0.1238 - avg_multilabel_BA_2: 0.8600 - val_loss: 0.1292 - val_avg_multilabel_BA_2: 0.8600
Epoch 51/100
3922/3922 [==============================] - 4s 928us/step - loss: 0.1236 - avg_multilabel_BA_2: 0.8602 - val_loss: 0.1287 - val_avg_multilabel_BA_2: 0.8602
Epoch 52/100
3922/3922 [==============================] - 4s 900us/step - loss: 0.1235 - avg_multilabel_BA_2: 0.8603 - val_loss: 0.1290 - val_avg_multilabel_BA_2: 0.8605
Epoch 53/100
3922/3922 [==============================] - 3s 887us/step - loss: 0.1234 - avg_multilabel_BA_2: 0.8606 - val_loss: 0.1293 - val_avg_mult

3922/3922 [==============================] - 3s 879us/step - loss: 0.1196 - avg_multilabel_BA_2: 0.8662 - val_loss: 0.1271 - val_avg_multilabel_BA_2: 0.8662
Epoch 97/100
3922/3922 [==============================] - 3s 887us/step - loss: 0.1197 - avg_multilabel_BA_2: 0.8662 - val_loss: 0.1266 - val_avg_multilabel_BA_2: 0.8663
Epoch 98/100
3922/3922 [==============================] - 3s 890us/step - loss: 0.1196 - avg_multilabel_BA_2: 0.8664 - val_loss: 0.1265 - val_avg_multilabel_BA_2: 0.8664
Epoch 99/100
3922/3922 [==============================] - 3s 885us/step - loss: 0.1194 - avg_multilabel_BA_2: 0.8664 - val_loss: 0.1270 - val_avg_multilabel_BA_2: 0.8665
Epoch 100/100
3922/3922 [==============================] - 3s 876us/step - loss: 0.1194 - avg_multilabel_BA_2: 0.8665 - val_loss: 0.1253 - val_avg_multilabel_BA_2: 0.8666
Best epoch: 100
Epoch 1/100
3922/3922 [==============================] - 5s 968us/step - loss: 0.1936 - avg_multilabel_BA_2: 0.8661 - val_loss: 0.1706 - val_avg_m

3922/3922 [==============================] - 3s 859us/step - loss: 0.1271 - avg_multilabel_BA_2: 0.8640 - val_loss: 0.1317 - val_avg_multilabel_BA_2: 0.8640
Epoch 45/100
3922/3922 [==============================] - 3s 875us/step - loss: 0.1269 - avg_multilabel_BA_2: 0.8640 - val_loss: 0.1316 - val_avg_multilabel_BA_2: 0.8640
Epoch 46/100
3922/3922 [==============================] - 3s 866us/step - loss: 0.1267 - avg_multilabel_BA_2: 0.8640 - val_loss: 0.1316 - val_avg_multilabel_BA_2: 0.8640
Epoch 47/100
3922/3922 [==============================] - 3s 870us/step - loss: 0.1267 - avg_multilabel_BA_2: 0.8640 - val_loss: 0.1311 - val_avg_multilabel_BA_2: 0.8640
Epoch 48/100
3922/3922 [==============================] - 4s 894us/step - loss: 0.1264 - avg_multilabel_BA_2: 0.8640 - val_loss: 0.1321 - val_avg_multilabel_BA_2: 0.8641
Epoch 49/100
3922/3922 [==============================] - 4s 912us/step - loss: 0.1264 - avg_multilabel_BA_2: 0.8640 - val_loss: 0.1325 - val_avg_multilabel_BA_2: 

3922/3922 [==============================] - 3s 851us/step - loss: 0.1230 - avg_multilabel_BA_2: 0.8654 - val_loss: 0.1299 - val_avg_multilabel_BA_2: 0.8654
Epoch 93/100
3922/3922 [==============================] - 3s 854us/step - loss: 0.1229 - avg_multilabel_BA_2: 0.8655 - val_loss: 0.1275 - val_avg_multilabel_BA_2: 0.8655
Epoch 94/100
3922/3922 [==============================] - 3s 853us/step - loss: 0.1228 - avg_multilabel_BA_2: 0.8655 - val_loss: 0.1316 - val_avg_multilabel_BA_2: 0.8655
Epoch 95/100
3922/3922 [==============================] - 3s 853us/step - loss: 0.1227 - avg_multilabel_BA_2: 0.8655 - val_loss: 0.1285 - val_avg_multilabel_BA_2: 0.8655
Epoch 96/100
3922/3922 [==============================] - 3s 861us/step - loss: 0.1228 - avg_multilabel_BA_2: 0.8655 - val_loss: 0.1279 - val_avg_multilabel_BA_2: 0.8656
Epoch 97/100
3922/3922 [==============================] - 3s 852us/step - loss: 0.1227 - avg_multilabel_BA_2: 0.8656 - val_loss: 0.1289 - val_avg_multilabel_BA_2: 

...

INFO:tensorflow:Oracle triggered exit


Epoch 1/100
3801/3801 [==============================] - 4s 944us/step - loss: 0.1974 - avg_multilabel_BA_2: 0.8672 - val_loss: 0.1696 - val_avg_multilabel_BA_2: 0.8671
Epoch 2/100
3801/3801 [==============================] - 3s 863us/step - loss: 0.1631 - avg_multilabel_BA_2: 0.8671 - val_loss: 0.1569 - val_avg_multilabel_BA_2: 0.8671
Epoch 3/100
3801/3801 [==============================] - 3s 857us/step - loss: 0.1552 - avg_multilabel_BA_2: 0.8670 - val_loss: 0.1523 - val_avg_multilabel_BA_2: 0.8670
Epoch 4/100
3801/3801 [==============================] - 3s 863us/step - loss: 0.1505 - avg_multilabel_BA_2: 0.8670 - val_loss: 0.1497 - val_avg_multilabel_BA_2: 0.8670
Epoch 5/100
3801/3801 [==============================] - 3s 860us/step - loss: 0.1473 - avg_multilabel_BA_2: 0.8670 - val_loss: 0.1456 - val_avg_multilabel_BA_2: 0.8670
Epoch 6/100
3801/3801 [==============================] - 3s 859us/step - loss: 0.1447 - avg_multilabel_BA_2: 0.8669 - val_loss: 0.1439 - val_avg_multilabel

3801/3801 [==============================] - 3s 873us/step - loss: 0.1275 - avg_multilabel_BA_2: 0.8667 - val_loss: 0.1307 - val_avg_multilabel_BA_2: 0.8668
Epoch 50/100
3801/3801 [==============================] - 3s 877us/step - loss: 0.1275 - avg_multilabel_BA_2: 0.8667 - val_loss: 0.1301 - val_avg_multilabel_BA_2: 0.8668
Epoch 51/100
3801/3801 [==============================] - 3s 864us/step - loss: 0.1273 - avg_multilabel_BA_2: 0.8667 - val_loss: 0.1325 - val_avg_multilabel_BA_2: 0.8668
Epoch 52/100
3801/3801 [==============================] - 3s 864us/step - loss: 0.1272 - avg_multilabel_BA_2: 0.8667 - val_loss: 0.1296 - val_avg_multilabel_BA_2: 0.8668
Epoch 53/100
3801/3801 [==============================] - 3s 859us/step - loss: 0.1272 - avg_multilabel_BA_2: 0.8667 - val_loss: 0.1316 - val_avg_multilabel_BA_2: 0.8668
Epoch 54/100
3801/3801 [==============================] - 3s 860us/step - loss: 0.1270 - avg_multilabel_BA_2: 0.8667 - val_loss: 0.1304 - val_avg_multilabel_BA_2: 

3801/3801 [==============================] - 3s 864us/step - loss: 0.1240 - avg_multilabel_BA_2: 0.8669 - val_loss: 0.1269 - val_avg_multilabel_BA_2: 0.8670
Epoch 98/100
3801/3801 [==============================] - 3s 861us/step - loss: 0.1241 - avg_multilabel_BA_2: 0.8669 - val_loss: 0.1272 - val_avg_multilabel_BA_2: 0.8670
Epoch 99/100
3801/3801 [==============================] - 3s 855us/step - loss: 0.1241 - avg_multilabel_BA_2: 0.8670 - val_loss: 0.1290 - val_avg_multilabel_BA_2: 0.8670
Epoch 100/100
3801/3801 [==============================] - 3s 860us/step - loss: 0.1240 - avg_multilabel_BA_2: 0.8670 - val_loss: 0.1292 - val_avg_multilabel_BA_2: 0.8670
Best epoch: 1
1856/1856 [==============================] - 1s 567us/step - loss: 0.1638 - avg_multilabel_BA_2: 0.8669
Test results - Loss: 0.16381414234638214 - Averaged Balanced Accuracy: 0.866909921169281%
Averaged Balanced Accuracy: 0.866877
INFO:tensorflow:Assets written to: model/saved_model_fold_4/assets


INFO:tensorflow:Assets written to: model/saved_model_fold_4/assets
2022-03-25 10:56:08.020262: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-03-25 10:56:08.020283: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-03-25 10:56:08.020287: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-03-25 10:56:08.020462: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model_fold_4
2022-03-25 10:56:08.021248: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-03-25 10:56:08.021260: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model_fold_4
2022-03-25 10:56:08.023502: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-03-25 10:56:08.050909: I tensorflow/cc/saved_model/loader.

Com os modelos salvos, o restante do experimento segue em ambiente Android